This notebook was inspired by this Anthropic's blog post:
- https://www.anthropic.com/research/building-effective-agents

and this source code:
- https://github.com/anthropics/anthropic-cookbook/blob/main/patterns/agents/basic_workflows.ipynb

In this notebook we will implement the essential patterns for working with LLMs that serve as a great introduction to LangChain components!

In [ ]:
%pip install openai>1.50.0 langchain>0.3.0 langgraph langchainhub langchain-openai langchain-community langchain-cli langchain_ollama tavily-python>=0.5.0 langchain_nomic nomic[local] langserve faiss-cpu tiktoken pypdf chroma jira google-search-results numexpr beautifulsoup4 scikit-learn

In [1]:
from openai import OpenAI
client = OpenAI()

# Chain Pattern

In [2]:
def llm_call(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def chain(input, prompts):
    result = input
    for i, prompt in prompts:
        print(f"Step: {i}")
        result = llm_call(f"{prompt}: \n\n{result}")
        print(result)
    
    return result

In [3]:
# Example prompts for a multi-step analysis chain
analysis_prompts = [
    (1, "Analyze this text and identify the main themes and topics"),
    (2, "For each theme identified, provide specific examples from the text"),
    (3, "Synthesize the themes and examples into a coherent summary"),
    (4, "Generate 3 discussion questions based on the analysis")
]

# Sample input text
sample_text = """
The rise of artificial intelligence has transformed multiple industries in the past decade. 
Healthcare has seen improvements in diagnosis accuracy and treatment planning. 
Manufacturing has become more efficient with predictive maintenance and automated quality control.
However, these advancements also raise important ethical questions about privacy, job displacement, 
and the role of human decision-making in an increasingly automated world.
"""

# Run the chain
print("Running analysis chain...")
final_result = chain(sample_text, analysis_prompts)
print(final_result)

Running analysis chain...
Step: 1
The text presents several main themes and topics regarding the impact of artificial intelligence (AI) across various sectors. Here are the key themes identified:

1. **Impact of Artificial Intelligence**: The text emphasizes the transformative effects of AI across multiple industries over the past decade.

2. **Improvements in Healthcare**: It highlights specific benefits of AI in healthcare, particularly in enhancing diagnosis accuracy and treatment planning.

3. **Efficiency in Manufacturing**: The text mentions efficiency gains in manufacturing, focusing on predictive maintenance and automated quality control processes.

4. **Ethical Considerations**: The text raises important ethical questions associated with the rise of AI, including:
   - **Privacy**: Concerns related to data privacy and the handling of personal information by AI systems.
   - **Job Displacement**: The potential impact of automation on employment and the workforce.
   - **Human D

# Parallel Pattern

In [4]:
from concurrent.futures import ThreadPoolExecutor

In [5]:
def parallel(prompt, inputs, n_workers=3):
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}: \n {input}") for input in inputs]
    
    return [f.result() for f in futures]


prompt = "Summarize this paragraph in one sentence"
inputs = [
    """
    Climate change is causing rising sea levels and extreme weather patterns globally. 
    Scientists have observed accelerating ice melt in polar regions, leading to coastal 
    flooding in low-lying areas. Additionally, communities worldwide are experiencing 
    more frequent and severe hurricanes, droughts, and other extreme weather events. 
    These changes pose significant risks to agriculture, infrastructure, and human 
    populations, particularly in vulnerable coastal regions and developing nations.
    """,
    """
    The development of quantum computers represents a major technological breakthrough
    with far-reaching implications. These powerful machines leverage quantum mechanical
    properties to perform complex calculations exponentially faster than classical computers.
    In the field of cryptography, quantum computers could break many current encryption
    methods while enabling new unbreakable encryption protocols. For drug discovery,
    quantum computers could simulate molecular interactions with unprecedented accuracy,
    potentially accelerating the development of new medicines and treatments for diseases.
    """,
    """
    Social media platforms have transformed how people communicate and share information
    in the modern digital age. Platforms like Facebook, Twitter, and Instagram have created
    virtual communities where users can instantly connect with friends and family across the
    globe. These platforms enable the rapid spread of news, ideas, and cultural trends,
    while also raising concerns about privacy, misinformation, and the impact on mental
    health. The rise of social media has also revolutionized marketing, activism, and
    how businesses engage with their customers.
    """
]

parallel(prompt, inputs)

['Climate change is resulting in rising sea levels and more severe weather events, which threaten agriculture, infrastructure, and populations, especially in coastal and developing areas.',
 'Quantum computers are a significant technological advancement that could revolutionize fields like cryptography and drug discovery by performing calculations much faster than classical computers and enabling new encryption methods and accurate molecular simulations.',
 'Social media platforms have revolutionized communication and information sharing, creating global connections while also raising issues related to privacy, misinformation, and mental health, and transforming marketing and activism.']

# Router Pattern

In [6]:
from enum import Enum
from pydantic import BaseModel, Field 
from typing import List


class ProfileRouter(BaseModel):
    selected_profile: str = Field(description="You return one of 3 profiles: [hr, software engineer, product manager]")
    

def llm_call_route(problem):
    '''Selects which profile to use to analyse the input.'''
    response = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        messages=[{'role': 'system', 'content': """
                   You select which of the following profiles should be used to solve a problem given as input:
                   1. hr
                   2. software engineer
                   3. product manager
                   Your output is solely the choosen profile string.
                   """},
                  {'role': 'user', 'content': f"We had this problem: \n {problem}. Output the appropriate profile to solve it:\n"}],
        response_format=ProfileRouter
    )
    return response.choices[0].message.parsed.selected_profile
    

llm_call_route("I need to fix this bug in our codebase")

'software engineer'

In [8]:
from typing import Dict


def router(input, routes: Dict[str, str]):
    
    route_response = llm_call_route(input)
    
    selected_prompt = routes[route_response]
    
    return llm_call(selected_prompt + f"Input: {input}")       

In [9]:
routes_profiles = {
    'hr': """You are an experienced HR professional focused on employee relations, recruitment, and workplace policies.
    Your expertise includes conflict resolution, talent acquisition, performance management, and ensuring compliance with labor laws.
    Analyze the input from an HR perspective and provide appropriate guidance.""",
    
    'software engineer': """You are a senior software engineer with deep technical expertise in software development, architecture, and best practices.
    Your skills include debugging, code optimization, system design, and technical problem-solving.
    Analyze the input from an engineering perspective and provide technical solutions.""",
    
    'product manager': """You are a seasoned product manager skilled in product strategy, user experience, and market analysis.
    Your expertise includes feature prioritization, roadmap planning, stakeholder management, and data-driven decision making.
    Analyze the input from a product perspective and provide strategic recommendations."""
}

problems = [
    "We need to improve our onboarding process for new employees to increase retention",
    "The authentication service is throwing intermittent 500 errors in production",
    "Our user engagement metrics have dropped 20% since the last release"
]

# Test the router with each problem
for problem in problems:
    print(f"\nProblem: {problem}")
    print(f"Response: {router(problem, routes_profiles)}")


Problem: We need to improve our onboarding process for new employees to increase retention
Response: Improving the onboarding process is a crucial step in enhancing employee retention. Here are several best practices and strategies you can implement to revamp your onboarding process effectively:

### 1. **Pre-Onboarding Engagement:**
   - **Welcome Kits:** Send a welcome email with essential information and a welcome kit that includes company swag, an employee handbook, and any necessary forms to complete.
   - **Connect Before Day One:** Assign a mentor or buddy to new hires before they start. This can help to foster relationships and ease first-day anxiety.

### 2. **Structured Onboarding Program:**
   - **Create a Comprehensive Onboarding Timeline:** Develop a structured program that spans at least 90 days. Include milestones for the first week, first month, and first three months.
   - **Clear Curriculum:** Ensure that the onboarding includes training on company culture, policies,

Now, that've learned these 3 basic patterns let's look at them implemented with LangChain to understand what LangChain tries to introduce with it's framework.

# Chains in LangChain

Now, first thing to note before reproducing the chain pattern in langchain is nomenclature, in LangChain, they define chain
as a composable building block built on top of a specialized interface named: [`Runnable Interface`](https://python.langchain.com/docs/concepts/runnables/), so instead of it being this action of chaining things together, it's the chainable thing itself!

Now, the concept remains the same:

- Composing calls to an LLM

Let's see how that would be done with LangChain.

Before we wrote:

In [ ]:
def llm_call(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def chain(input, prompts):
    result = input
    for i, prompt in prompts:
        print(f"Step: {i}")
        result = llm_call(f"{prompt}: \n\n{result}")
        print(result)
    
    return result

- The `llm_call()` function calls the LLM simply.
- The `chain` function takes in an input and a list of prompts and runs all the prompts on that input in sequence.

<!-- This approach means that if we wanted each prompt for example to be used with a different LLM, that would require rewriting the implementations.

LangChain assumes this flexibility from the start! -->

In [4]:
import os
# os.environ["LANGCHAIN_API_KEY"] =
os.environ["LANGCHAIN_TRACING_V2"] = "true" 
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="default"  # if not specified, 

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def llm_call(prompt):
    return ChatOpenAI(model="gpt-4o-mini").invoke(prompt).content

llm_call("hi")

'Hello! How can I assist you today?'

In [11]:
analysis_prompts = [
    (1, "Analyze this text and identify the main themes and topics"),
    (2, "For each theme identified, provide specific examples from the text"),
    (3, "Synthesize the themes and examples into a coherent summary"),
    (4, "Generate 3 discussion questions based on the analysis")
]

# Sample input text
sample_text = """
The rise of artificial intelligence has transformed multiple industries in the past decade. 
Healthcare has seen improvements in diagnosis accuracy and treatment planning. 
Manufacturing has become more efficient with predictive maintenance and automated quality control.
However, these advancements also raise important ethical questions about privacy, job displacement, 
and the role of human decision-making in an increasingly automated world.
"""

prompt1 = ChatPromptTemplate.from_template("Analyze this text and identify the main themes and topics: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
chain1 = prompt1 | llm

prompt2 = ChatPromptTemplate.from_template("For each theme identified, provide specific examples from the text: {input}")
chain2 = prompt2 | llm

prompt3 = ChatPromptTemplate.from_template("Synthesize the themes and examples into a coherent summary: {input}")
chain3 = prompt3 | llm

prompt4 = ChatPromptTemplate.from_template("Generate 3 discussion questions based on the analysis: {input}")
chain4 = prompt4 | llm

final_chain = chain1 | chain2 | chain3 | chain4

output = final_chain.invoke({"input": sample_text})
output

AIMessage(content='Here are three discussion questions based on the analysis of the transformative impact of AI and its accompanying ethical concerns:\n\n1. **Balancing Innovation and Ethics**: How can industries leverage the benefits of AI, such as improved efficiency and accuracy, while simultaneously addressing the ethical concerns related to privacy and job displacement? What frameworks or guidelines could be implemented to ensure responsible AI integration?\n\n2. **Human vs. Machine Decision-Making**: In what ways can organizations maintain a balance between human judgment and AI-driven automation in decision-making processes? What are the potential risks of over-relying on AI in critical sectors, such as healthcare and manufacturing?\n\n3. **Future Workforce and AI Integration**: As AI continues to evolve and automate tasks, what strategies should be adopted to prepare the workforce for the changing job landscape? How can educational institutions and businesses collaborate to ups

We could simplify this by writing a function to abstract the creation of each chain block (runnable).

In [12]:
from langchain_core.runnables import RunnableSequence

def create_runnable_chain(prompt):
    return ChatPromptTemplate.from_template(prompt) | llm

In [20]:
# Example usage of create_runnable_chain
prompts = [
    "Analyze this text and identify the main themes and topics: {input}",
    "For each theme identified, provide specific examples from the text: {input}",
    "Synthesize the themes and examples into a coherent summary: {input}",
    "Generate 3 discussion questions based on the analysis: {input}"
]

# Create individual chains
chains = [create_runnable_chain(prompt) for prompt in prompts]

# Combine chains into a sequence
# Below is equivalent to: final_chain = chain1 | chain2 | ...
final_chain = RunnableSequence(*chains)

# Sample input text
sample_text = """
The rise of artificial intelligence has transformed multiple industries in the past decade.
Healthcare has seen improvements in diagnosis accuracy and treatment planning.
Manufacturing has become more efficient with predictive maintenance and automated quality control.
However, these advancements also raise important ethical questions about privacy, job displacement,
and the role of human decision-making in an increasingly automated world.
"""

# Run the chain
output = final_chain.invoke({"input": sample_text})
print(output)

content='Here are three discussion questions based on the analysis of the impact of artificial intelligence (AI):\n\n1. **Transformation of Industries**: How do you envision the ongoing transformation of industries by AI will shape the workforce in the next decade? What skills do you think will become essential for workers in sectors heavily influenced by AI, such as healthcare and manufacturing?\n\n2. **Ethical Considerations**: In light of the ethical challenges posed by AI, particularly regarding privacy and job displacement, what measures should organizations and governments take to ensure that the benefits of AI are distributed equitably while minimizing harm to individuals and communities?\n\n3. **Human Decision-Making**: As AI systems become increasingly integrated into decision-making processes, what safeguards can be implemented to maintain human oversight and accountability? How can we ensure that human judgment is not entirely supplanted by automated systems, especially in c

Another option would be to use the convenient `@chain` decorator.

In [5]:
from langchain_core.runnables import chain

@chain
def custom_chain(text):
    # Analyze themes
    analyze_chain = ChatPromptTemplate.from_template("Analyze this text and identify the main themes and topics: {input}") | llm
    themes = analyze_chain.invoke({"input": text})
    
    # Get examples
    examples_chain = ChatPromptTemplate.from_template("For each theme identified, provide specific examples from the text: {input}") | llm
    examples = examples_chain.invoke({"input": themes.content})
    
    # Synthesize
    synthesis_chain = ChatPromptTemplate.from_template("Synthesize the themes and examples into a coherent summary: {input}") | llm
    synthesis = synthesis_chain.invoke({"input": examples.content})
    
    # Generate questions
    questions_chain = ChatPromptTemplate.from_template("Generate 3 discussion questions based on the analysis: {input}") | llm
    return questions_chain.invoke({"input": synthesis.content})

In [7]:
sample_text = """
The rise of artificial intelligence has transformed multiple industries in the past decade.
Healthcare has seen improvements in diagnosis accuracy and treatment planning.
Manufacturing has become more efficient with predictive maintenance and automated quality control.
However, these advancements also raise important ethical questions about privacy, job displacement,
and the role of human decision-making in an increasingly automated world.
"""
llm = ChatOpenAI(model='gpt-4o-mini')
custom_chain.invoke(sample_text)

AIMessage(content='Here are three discussion questions based on the analysis of the themes surrounding the rise of artificial intelligence (AI):\n\n1. **Balancing Innovation and Ethics**: Considering the transformative impact of AI across industries such as healthcare and manufacturing, how can organizations implement AI technologies while prioritizing ethical considerations, particularly in relation to privacy and potential job displacement? What frameworks or guidelines could be established to ensure responsible use of AI?\n\n2. **Human Oversight in AI Applications**: In light of the potential biases in AI algorithms that could lead to harmful outcomes, especially in critical areas like criminal justice and healthcare, what measures should be taken to ensure that human decision-making remains central in the deployment of AI systems? How can organizations effectively integrate human oversight without undermining the efficiency that AI offers?\n\n3. **Workforce Adaptation to AI Integra

# Parallel

Our previous implementation:

In [ ]:
def parallel(prompt, inputs, n_workers=3):
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}: \n {input}") for input in inputs]
    
    return [f.result() for f in futures]

Now with langchain:

In [13]:
from langchain_core.runnables import RunnableParallel

prompt_template = "Summarize this paragraph in one sentence: {input}"
inputs = [
    """
    Climate change is causing rising sea levels and extreme weather patterns globally. 
    Scientists have observed accelerating ice melt in polar regions, leading to coastal 
    flooding in low-lying areas. Additionally, communities worldwide are experiencing 
    more frequent and severe hurricanes, droughts, and other extreme weather events. 
    These changes pose significant risks to agriculture, infrastructure, and human 
    populations, particularly in vulnerable coastal regions and developing nations.
    """,
    """
    The development of quantum computers represents a major technological breakthrough
    with far-reaching implications. These powerful machines leverage quantum mechanical
    properties to perform complex calculations exponentially faster than classical computers.
    In the field of cryptography, quantum computers could break many current encryption
    methods while enabling new unbreakable encryption protocols. For drug discovery,
    quantum computers could simulate molecular interactions with unprecedented accuracy,
    potentially accelerating the development of new medicines and treatments for diseases.
    """,
    """
    Social media platforms have transformed how people communicate and share information
    in the modern digital age. Platforms like Facebook, Twitter, and Instagram have created
    virtual communities where users can instantly connect with friends and family across the
    globe. These platforms enable the rapid spread of news, ideas, and cultural trends,
    while also raising concerns about privacy, misinformation, and the impact on mental
    health. The rise of social media has also revolutionized marketing, activism, and
    how businesses engage with their customers.
    """
]

prompt = ChatPromptTemplate.from_template(prompt_template)

chain = prompt | llm 

chain.batch([{"input":input} for input in inputs])

[AIMessage(content='Climate change is resulting in rising sea levels and more severe weather events, threatening agriculture, infrastructure, and vulnerable populations, especially in low-lying and developing areas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 106, 'total_tokens': 139, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-5aff0776-2f6f-4888-80ce-28baaebaf503-0', usage_metadata={'input_tokens': 106, 'output_tokens': 33, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Quantum computers signify a significant technological a

# Router Pattern

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

# Define classification chain
classification_chain = (
    PromptTemplate.from_template(
        """Given the user problem below, classify it as either `HR`, `Software Engineer`, or `Product Manager`.

Do not respond with more than one word.

<problem>
{problem}
</problem>

Classification:"""
    )
    | ChatOpenAI(model_name="gpt-4o-mini")
    | StrOutputParser()
)

# Define sub-chains
hr_chain = PromptTemplate.from_template(
    """You are an experienced HR professional focused on employee relations, recruitment, and workplace policies.
Your expertise includes conflict resolution, talent acquisition, performance management, and ensuring compliance with labor laws.
Analyze the input from an HR perspective and provide appropriate guidance.

Problem: {problem}
Solution:"""
) | ChatOpenAI(model_name="gpt-4o-mini")

software_engineer_chain = PromptTemplate.from_template(
    """You are a senior software engineer with deep technical expertise in software development, architecture, and best practices.
Your skills include debugging, code optimization, system design, and technical problem-solving.
Analyze the input from an engineering perspective and provide technical solutions.

Problem: {problem}
Solution:"""
) | ChatOpenAI(model_name="gpt-4o-mini")

product_manager_chain = PromptTemplate.from_template(
    """You are a seasoned product manager skilled in product strategy, user experience, and market analysis.
Your expertise includes feature prioritization, roadmap planning, stakeholder management, and data-driven decision-making.
Analyze the input from a product perspective and provide strategic recommendations.

Problem: {problem}
Solution:"""
) | ChatOpenAI(model_name="gpt-4o-mini")

# Define routing function
def route(info):
    if "hr" in info["category"].lower():
        print("Classified as HR.")
        return hr_chain
    elif "software engineer" in info["category"].lower():
        print("Classified as Software Engineer.")
        return software_engineer_chain
    elif "product manager" in info["category"].lower():
        print("Classified as Product Manager.")
        return product_manager_chain
    else:
        return product_manager_chain  # Default fallback

# Create runnable pipeline
full_chain = {"category": classification_chain, "problem": lambda x: x["problem"]} | RunnableLambda(route)

# Example problems
problems = [
    "We need to improve our onboarding process for new employees to increase retention",
    "The authentication service is throwing intermittent 500 errors in production",
    "Our user engagement metrics have dropped 20% since the last release"
]

# Invoke the chain for each problem
for problem in problems:
    response = full_chain.invoke({"problem": problem})
    print(f"Problem: {problem}\nSolution: {response}\n{'-' * 50}")


Classified as HR.
Problem: We need to improve our onboarding process for new employees to increase retention
Solution: content="Improving the onboarding process is crucial for enhancing employee retention, as a well-structured onboarding program can significantly impact new hires' engagement, satisfaction, and overall integration into the company culture. Here are several strategic recommendations to enhance your onboarding process from an HR perspective:\n\n### 1. **Pre-Onboarding Preparation**\n   - **Clear Communication:** Send a welcome email before the start date that outlines what new hires can expect on their first day, including schedules, dress codes, and required documentation.\n   - **Resource Access:** Provide access to digital resources (e.g., employee handbooks, organizational charts, and training materials) before their first day to allow new hires to familiarize themselves with the company.\n\n### 2. **Structured Onboarding Program**\n   - **Comprehensive Orientation:**

The only new concept here is that we used:
[`RunnableLambda`](https://arc.net/l/quote/ovwpozka)
to setup our custom logic.

In [15]:
response

AIMessage(content='When faced with a 20% drop in user engagement metrics post-release, it is crucial to conduct a thorough analysis to identify the root causes and implement strategic recommendations. Here’s a structured approach to address the issue from a product management perspective:\n\n### 1. Analyze User Engagement Data\n- **Metrics Breakdown**: Analyze specific engagement metrics (session duration, frequency of use, active users, etc.) to determine which areas have been most affected.\n- **Segment Users**: Identify if certain user segments (e.g., new vs. returning users, different demographics, or user personas) are more affected than others.\n- **Behavioral Analysis**: Use tools like heatmaps and user recordings to understand how users are interacting with the product post-release.\n\n### 2. Conduct User Research\n- **Surveys and Feedback**: Deploy surveys or feedback forms to gather qualitative data about user experience and perceived value.\n- **Usability Testing**: Conduct 